In [ ]:
import pydeb
import numpy

In [ ]:
# Obtain Catalogue of Life identifier for the taxon of interest
# This function (get_ids) will return multiple matches if the provided name is not unique.
pydeb.infer.get_ids('Hamiota altilis')

In [ ]:
# Obtain the prior distribution of DEB parameters through phylogenetic inference
taxon = pydeb.infer.Taxon.from_col_id('f663015430cb783a8c3c9bd4acf5ad98')
prior = taxon.infer_parameters()

In [ ]:
# Body temperature to use for prior information and simulations
T = taxon.typical_temperature

# Add observed life history traits (ultimate structural length and reproduction rate)
observations = []
observations.append(pydeb.calibrate.likelihood.ImpliedProperty('L_i', numpy.log10(2.), 0.01, numpy.log10, temperature=T))
observations.append(pydeb.calibrate.likelihood.ImpliedProperty('R_i', numpy.log10(1000.), 0.01, numpy.log10, temperature=T))

sim = pydeb.calibrate.EnsembleRunner(prior.names, prior.inverse_transforms, prior.mean, prior.cov, sample_size=10000, deb_type='abj', temperature=T, priors=observations, selected_outputs=('L', 'R', 'S'))
display(sim.get_progress_bar())
sim.join()

In [ ]:
result = sim.get_result()

import plotly.offline as py
import plotly.graph_objects as go

for name in sim.selected_outputs:
    values = result[name]
    title = '%s (%s)' % (pydeb.long_names[name], pydeb.units[name])
    data = [
        go.Scatter(x=result['t'], y=values[0], name='2.5%', line={'color': 'blue'}),
        go.Scatter(x=result['t'], y=values[4], name='97.5%', line={'color': 'blue'}, fill='tonexty'),
        go.Scatter(x=result['t'], y=values[1], name='25%', line={'color': 'red'}),
        go.Scatter(x=result['t'], y=values[3], name='50%', line={'color': 'red'}, fill='tonexty'),
        go.Scatter(x=result['t'], y=values[2], name='median', line={'color': 'black'}),
    ]
    layout = {'xaxis': {'title': 'time (d)'}, 'yaxis': {'title': title}}
    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig)